# Test IE functions

In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerlib.span import Span
from spannerlib.session import Session,test_session

In [ ]:
def yield_range(num: int):
        for i in range(num):
            yield i,

    
def yield_range_str(num: int):
    for i in range(num):
        yield "string" + str(i),


test_session(
    [
        """
        test_range(X) <- yield_range(5) -> (X)
        ?test_range(X)
        """,

        """
        test_range_str(X) <- yield_range_str(5) -> (X)
        ?test_range_str(X)
        """,
    ],
    [
        pd.DataFrame({'X':range(5)}),
        pd.DataFrame({'X':["string0","string1","string2","string3","string4"]}),
    ],
    ie_funcs=[
        ['yield_range',yield_range,[int],[int]],
        ['yield_range_str',yield_range_str,[int],[str]]
    ],
)


'?test_range(X)'

X
0  0
1  1
2  2
3  3
4  4

'?test_range_str(X)'

X
0  string0
1  string1
2  string2
3  string3
4  string4

In [ ]:
sess = test_session(
[
    """
    py_string_rel(X) <- rgx("aa",".+") -> (X)
    py_span_rel_raw(X) <- rgx_span("aa",".+") -> (X)
    py_span_rel(X) <- rgx_span("aa",".+") -> (S), as_str(S)->(X)
    ?py_string_rel(X)
    """,
    """
    ?py_span_rel(X)
    """
],
[
    pd.DataFrame([
        ['aa'],
    ],columns=['X']),
    pd.DataFrame([
        ['aa'],
    ],columns=['X']),
],
ret_sess=True,
)
_ = sess.export('?py_span_rel_raw(X)',display_results=True)

'?py_string_rel(X)'

X
0  aa

'?py_span_rel(X)'

X
0  aa

'?py_span_rel_raw(X)'

X
0  [@e0c903,0,2) "aa"

In [ ]:
# uncomment these once we decide how to support rust regexs

In [ ]:
# test_session(
# [
#     """
#     string_rel(X) <- rgx_string("aa",".+") -> (X)
#     span_rel(X) <- rgx_span("aa",".+") -> (X)
#     ?string_rel(X)
#     """
#     """
#     ?span_rel(X)
#     """
# ],[
#     pd.DataFrame([
#         ['a'],
#         ['aa'],
#     ],columns=['X']),
#     pd.DataFrame([
#         [Span(0,1)],
#         [Span(0,2)],
#         [Span(1,2)]
#     ],columns=['X']),
# ],
# )

In [ ]:
# with tempfile.TemporaryDirectory() as temp_dir:
#         rgx_text_file = Path(temp_dir) / "temp"
#         with open(rgx_text_file, "w") as f:
#             f.write("aa")

# test_session(
# [
#     """
#     string_rel(X) <- rgx_string_from_file("tests/rgx_text_file",".+") -> (X)
#     span_rel(X) <- rgx_span_from_file("tests/rgx_text_file",".+") -> (X)
#     ?string_rel(X)
#     """
#     """
#     ?span_rel(X)
#     """
# ],[
#     pd.DataFrame([
#         ['a'],
#         ['aa'],
#     ],columns=['X']),
#     pd.DataFrame([
#         [Span(0,1)],
#         [Span(0,2)],
#         [Span(1,2)]
#     ],columns=['X']),
# ],
# )

In [ ]:
# text = "aab"
# pattern = "(?P<group_all>(?P<group_a>a+)(?P<group_b>b+))"
# test_session(
# [
#     f"""
#     group_string_rel(X,Y,Z) <- rgx_string("{text}","{pattern}") -> (X,Y,Z)
#     ?group_string_rel(X, Y, Z)
#     """
# ],[
#     pd.DataFrame([
#         ['aa','b','aab'],
#         ['a','b','ab'],
#     ],columns=['X','Y','Z']),
# ],
# )

In [ ]:
# TODO add tests for ie functions from stdlib, do this only for the ones we leave in the final demo version